In [ ]:
from snowflake.snowpark import Session
from snowflake.ml.fileset import sfcfs,fileset
import gzip
import pyarrow.parquet as pq
from torch.utils.data import DataLoader

In [ ]:
session=Session.builder.getOrCreate()

In [ ]:
session

In [ ]:
STAGE="@test.public.enc_stage"

## FileSystem

In [ ]:
fs=sfcfs.SFFileSystem(snowpark_session=session)

In [ ]:
fs.ls(STAGE)

In [ ]:
# reading a csv
with fs.open(f"{STAGE}/titanic.csv") as f:
    print(f.read(1000))

In [ ]:
# reading a gzip
with fs.open(f"{STAGE}/imdb_train.csv.gz", mode='rb') as f:
    g=gzip.GzipFile(fileobj=f)
    for i in range(10):
        print(g.readline())

In [ ]:
# Opening a Parquet File
df=pq.read_table(source=f"{STAGE}/weather-ts.parquet",filesystem=fs).to_pandas()
df.head()
    

## Fileset

In [ ]:
## Creating Fileset from a Dataframe
session.sql(f"REMOVE {STAGE}/housing1").collect()

In [ ]:
fs1=fileset.FileSet.make(
    target_stage_loc=STAGE,
    name='housing1',
    snowpark_dataframe=session.table("test.public.housing"),
    shuffle=True
)

In [ ]:
fs1.files()

In [ ]:
# Create a fileset from a query reult
session.sql(f"REMOVE {STAGE}/housing2").collect()

In [ ]:
fs2=fileset.FileSet.make(
    target_stage_loc=STAGE,
    name="housing2",
    sf_connection=session.connection,
    query="SELECT * FROM test.public.housing",
    shuffle=True
)

In [ ]:
fs2.files()

In [ ]:
# feed first fileset to tensorflow
ds1=fs1.to_tf_dataset(batch_size=4,shuffle=True,drop_last_batch=True)

In [ ]:
for batch in ds1:
    print(batch)
    break

In [ ]:
# Feed second file to Pytorch
s2=fs2.to_torch_datapipe(batch_size=4,shuffle=True, drop_last_batch=True)
loader=DataLoader(ds2,batch_size=None,num_workers=0)

In [ ]:
for batch in loader:
    print(batch)
    break